In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('practice').getOrCreate()

In [3]:
spark.version

'3.4.1'

In [5]:
strings = spark.read.text('README.md')

In [6]:
strings.show(10, truncate=False)
# 10줄을 보여줘!
# truncate=False: 문자열이 길어도 자르지 말고 보여줘!

+--------------------------------------------------------------------------------------------------+
|value                                                                                             |
+--------------------------------------------------------------------------------------------------+
|# Apache Spark                                                                                    |
|                                                                                                  |
|Spark is a unified analytics engine for large-scale data processing. It provides                  |
|high-level APIs in Scala, Java, Python, and R, and an optimized engine that                       |
|supports general computation graphs for data analysis. It also supports a                         |
|rich set of higher-level tools including Spark SQL for SQL and DataFrames,                        |
|pandas API on Spark for pandas workloads, MLlib for machine learning, GraphX for graph pro

In [9]:
filtered = strings.filter(strings.value.contains('Spark'))
filtered.count()

20

# 쿠키 몬스터를 위한 M&M 세기
- 여러 주(state) 출신 학생들에게 상으로 쿠키 제공
- 쿠키는 M&M을 사용하여 만들어짐
- 서로 다른 주에 사는 학생들에게 적절한 비율로 쿠키가 주어지는지 확인하고 싶다.

In [20]:
import sys
from pyspark.sql import SparkSession

if __name__ == '__main__':
    if len(sys.argv) != 2:
        print('Usage: mmcount <file>', file=sys.stderr)
        sys.exit(-1)
        
    # SparkSession API를 써서 SparkSession 객체를 만든다.
    # JVM마다 하나만 존재할 수 있다.
    spark = (SparkSession
            .builder
            .appName('PythonMnMCount')
            .getOrCreate())
    
    # 명령행 인자에서 M&M 데이터가 들어 있는 파일 이름을 얻는다.
    mnm_file = sys.argv[1]
    
    # 스키마 추론과 쉼표로 구분된 칼럼 이름이 제공되는 헤더가 있음을
    # 지정해주고 CSV 포맷으로 파일을 읽어 들여 데이터 프레임에 저장한다.
    mnm_df = (spark.read.format('csv')
             .option('header', 'true')
             .option('inferSchema', 'true')
             .load(mnm_file))
    
    count_mnm_df = (mnm_df
                   .select('State', 'Color', 'Count')
                   .groupBy('State', 'Color')
                   .sum('Count')
                   .orderBy('sum(Count)', ascending=False))
    
    # show()는 액션이므로 위의 쿼리 내용들이 시작되게 된다는 점에 주목하자.
    count_mnm_df.show(n=60, truncate=False)
    print('Total Rows = %d' % (count_mnm_df.count()))
    
    # 위 코드는 모든 주에 대한 집계를 계산한것이다.
    # 캘리포니아(CA)에 대해 보기를 원한다면?
    ca_count_mnm_df = (mnm_df
                       .select('State', 'Color', 'Count')
                       .where(mnm_df.State == 'CA')
                       .groupBy('State', 'Color')
                       .sum('Count')
                       .orderBy('sum(Count)', ascending=False))
    
    ca_count_mnm_df.show(n=10, truncate=False)
    
    # SparkSession을 멈춘다
    spark.stop()

Usage: mmcount <file>


SystemExit: -1

선호하는 편집기를 써서 mnmcount.py라는 이름의 파일을 만들어 위 파이썬 코드를 입력하고 mnm_dataset.csv를 다운로드해 bin 디렉터리의 submit-spark 스크립트로 스파크 잡을 제출하라는데 어떻게 하는지 모르겠음. 리눅스나 맥 OS로 진행하는 책이라 쥬피터노트북으로 진행하는데 제한이 있을듯 <br>
csv파일을 다운받고 직접 실행해본다.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('mnm').getOrCreate()

mnm_df = (spark.read.format('csv')
         .option('header', 'true')
         .option('inferSchema', 'true')
         .load('mnm_dataset.csv'))

count_mnm_df = (mnm_df
               .select('State', 'Color', 'Count')
               .groupBy('State', 'Color')
               .sum('Count')
               .orderBy('sum(Count)', ascending=False))

# show()는 액션이므로 위의 쿼리 내용들이 시작되게 된다는 점에 주목하자.
count_mnm_df.show(n=60, truncate=False)
print('Total Rows = %d' % (count_mnm_df.count()))

# 위 코드는 모든 주에 대한 집계를 계산한것이다.
# 캘리포니아(CA)에 대해 보기를 원한다면?
ca_count_mnm_df = (mnm_df
                   .select('State', 'Color', 'Count')
                   .where(mnm_df.State == 'CA')
                   .groupBy('State', 'Color')
                   .sum('Count')
                   .orderBy('sum(Count)', ascending=False))

ca_count_mnm_df.show(n=10, truncate=False)

# SparkSession을 멈춘다
spark.stop()

+-----+------+----------+
|State|Color |sum(Count)|
+-----+------+----------+
|CA   |Yellow|100956    |
|WA   |Green |96486     |
|CA   |Brown |95762     |
|TX   |Green |95753     |
|TX   |Red   |95404     |
|CO   |Yellow|95038     |
|NM   |Red   |94699     |
|OR   |Orange|94514     |
|WY   |Green |94339     |
|NV   |Orange|93929     |
|TX   |Yellow|93819     |
|CO   |Green |93724     |
|CO   |Brown |93692     |
|CA   |Green |93505     |
|NM   |Brown |93447     |
|CO   |Blue  |93412     |
|WA   |Red   |93332     |
|WA   |Brown |93082     |
|WA   |Yellow|92920     |
|NM   |Yellow|92747     |
|NV   |Brown |92478     |
|TX   |Orange|92315     |
|AZ   |Brown |92287     |
|AZ   |Green |91882     |
|WY   |Red   |91768     |
|AZ   |Orange|91684     |
|CA   |Red   |91527     |
|WA   |Orange|91521     |
|NV   |Yellow|91390     |
|UT   |Orange|91341     |
|NV   |Green |91331     |
|NM   |Orange|91251     |
|NM   |Green |91160     |
|WY   |Blue  |91002     |
|UT   |Red   |90995     |
|CO   |Orang